# Install libraries 


In [1]:
import sys, os, json
import mne, sklearn, wandb
import numpy as np
import pandas as pd

from scipy.interpolate import interp1d
from nilearn import datasets, image, masking, plotting
from nilearn.input_data import NiftiLabelsMasker


# animation part
from IPython.display import HTML
import matplotlib
import matplotlib.pyplot as plt
# from celluloid import Camera   # it is convinient method to animate
from matplotlib import animation, rc
from matplotlib.animation import FuncAnimation


## torch libraries 
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, TensorDataset
from torch.utils.data import DataLoader, Subset

from pytorch_model_summary import summary


/opt/conda/envs/myenv_torch/lib/python3.9/site-packages/nilearn/input_data/__init__.py:27: FutureWarning: The import path 'nilearn.input_data' is deprecated in version 0.9. Importing from 'nilearn.input_data' will be possible at least until release 0.13.0. Please import from 'nilearn.maskers' instead.
  warnings.warn(message, FutureWarning)


In [2]:
%load_ext autoreload
%autoreload 2
sys.path.insert(1, os.path.realpath(os.path.pardir))

from utils import get_datasets
from utils import preproc
from utils import torch_dataset
from utils import train_utils
from utils import inference
from utils.models_arch import autoencoder_new

# Set all hyperparameters
- Cuda and GPU.
- Parameters of dataset. 
- random seed( if necessary). 


In [3]:
# import random

# torch.manual_seed(0)
# random.seed(0)  # python operation seed
# np.random.seed(0)

# torch.backends.cudnn.benchmark = False
# torch.backends.cudnn.deterministic = True

print(torch.cuda.is_available(), torch.cuda.device_count())
torch.cuda.set_device(2)

True 4


In [4]:
config = dict(  
                dataset_name = 'CWL', # CWL
                new_fps=100, 
                freqs = np.logspace(np.log10(2), np.log10(99), 16), 
    
                n_channels = 30, # 30 
                n_roi = 6,
                
                bold_delay = 5,
                to_many = True,
                random_subsample = True,
                sample_per_epoch = 2048, 
                WINDOW_SIZE = 2048,
                    
                optimizer='adam',
                lr=3e-4,
                weight_decay=0.0001, 
                batch_size=4, 
                
                preproc_type = 'dB_log',
                loss_function = 'mse', 
                model_type = '1D_CNN_AE_wav2vec2'
                )


hp_autoencoder = dict(n_electrodes=config['n_channels'],
                      n_freqs = len(config['freqs']),
                      n_channels_out = config['n_roi'],

                     channels = [256, 256, 256, 256], 
                     kernel_sizes=[3, 3, 3],
                     strides=[8, 4, 4], 
                     dilation=[1, 1, 1], 
                     decoder_reduce=4
                     )


config = {**hp_autoencoder, **config}

params_train = {'batch_size': config['batch_size'],
                'shuffle': True,
                'num_workers': 0}

params_val = {'batch_size': config['batch_size'],
              'shuffle': False}

# Upload preprocessed dataset from np files. 
It should accelerate speed of experiments.

In [5]:
with open("../data/processed/labels_roi_6.json", 'r') as f:
    labels_roi = json.load(f)


if config['dataset_name']=='CWL':
    dataset_path = '../data/processed/CWL/trio1_100_hz_6_roi_2_99_freqs.npz'
    
elif config['dataset_name']=='NODDI':
    dataset_path = '../data/processed/NODDI/32_100_hz_6_roi_2_99_freqs.npz'
else:
    print('no such dataset')


# download data
data = np.load(dataset_path)

train_dataset_prep = (data['x_train'], data['y_train'])
test_dataset_prep = (data['x_test'], data['y_test'])


# apply time dealy corrected
train_dataset_prep = preproc.bold_time_delay_align(train_dataset_prep, 
                                                   config['new_fps'],
                                                   config['bold_delay'])
test_dataset_prep = preproc.bold_time_delay_align(test_dataset_prep, 
                                                  config['new_fps'],
                                                  config['bold_delay'])


print('Size of train dataset:', train_dataset_prep[0].shape, train_dataset_prep[1].shape)
print('Size of test dataset:', test_dataset_prep[0].shape, test_dataset_prep[1].shape)

# torch dataset creation 
torch_dataset_train = torch_dataset.CreateDataset_eeg_fmri(train_dataset_prep, 
                                                            random_sample=config['random_subsample'], 
                                                            sample_per_epoch=config['sample_per_epoch'], 
                                                            to_many=config['to_many'], 
                                                            window_size = config['WINDOW_SIZE'])

torch_dataset_test = torch_dataset.CreateDataset_eeg_fmri(test_dataset_prep, 
                                                            random_sample=False, 
                                                            sample_per_epoch=None, 
                                                            to_many=config['to_many'], 
                                                            window_size = config['WINDOW_SIZE'])


# init dataloaders for training
train_loader = torch.utils.data.DataLoader(torch_dataset_train, **params_train)
val_loader = torch.utils.data.DataLoader(torch_dataset_test, **params_val)




Size of train dataset: (30, 16, 20690) (6, 20690)
Size of test dataset: (30, 16, 5500) (6, 5500)


# Init Model, Loss, optimizers

In [6]:
model = autoencoder_new.AutoEncoder1D(**hp_autoencoder)

print(summary(model, torch.zeros(4, config['n_channels'], 
                                 len(config['freqs']),
                                 config['WINDOW_SIZE']), show_input=False))


-----------------------------------------------------------------------
      Layer (type)        Output Shape         Param #     Tr. Param #
       ConvBlock-1      [4, 256, 2048]         369,152         369,152
       ConvBlock-2       [4, 256, 256]         197,120         197,120
       ConvBlock-3        [4, 256, 64]         197,120         197,120
       ConvBlock-4        [4, 256, 16]         197,120         197,120
     UpConvBlock-5         [4, 64, 64]          49,280          49,280
     UpConvBlock-6        [4, 64, 256]          12,416          12,416
     UpConvBlock-7       [4, 64, 2048]          12,416          12,416
          Conv1d-8        [4, 6, 2048]             390             390
Total params: 1,035,014
Trainable params: 1,035,014
Non-trainable params: 0
-----------------------------------------------------------------------


# Model training

In [ ]:
n_runs = 3

for i in range(n_runs):
    
    model = autoencoder_new.AutoEncoder1D(**hp_autoencoder)

    loss_func = train_utils.make_mse_loss()
    train_step = train_utils.train_step

    optimizer = optim.Adam(model.parameters(), 
                       lr=config['lr'], 
                       weight_decay=config['weight_decay'])
    
    
    parameters = {
        'EPOCHS': 100,
        'model': model, 
        'train_loader': train_loader, 
        'val_loader': val_loader, 
        'loss_function': loss_func,
        'train_step': train_step,
        'optimizer': optimizer, 
        'device': 'cuda', 
        'raw_test_data': test_dataset_prep,
        'show_info': 5, 
        'num_losses': 5,
        'labels': labels_roi,
        'inference_function': inference.model_inference_function, 
        'to_many': config['to_many']
    }



    path_to_save_wandb = 'common/koval_alvi/Checkpoints/wandb_brain'
    
    
    with wandb.init(project="eeg_fmri", config=config, save_code=True):
        
        wandb.define_metric("val/corr_mean", summary="max")

        if i == 0: 
            exp_name = wandb.run.name
        
        wandb.run.name = exp_name +'_run_' + str(i)
        
        print(config)
        print(parameters['model'])
        print(summary(model, torch.zeros(4, config['n_channels'],
                                         len(config['freqs']), config['WINDOW_SIZE']), show_input=False))
        
        model = train_utils.wanb_train_regression(**parameters)
        

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: koval_alvi (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


{'n_electrodes': 30, 'n_freqs': 16, 'n_channels_out': 6, 'channels': [256, 256, 256, 256], 'kernel_sizes': [3, 3, 3], 'strides': [8, 4, 4], 'dilation': [1, 1, 1], 'decoder_reduce': 4, 'dataset_name': 'CWL', 'new_fps': 100, 'freqs': array([ 2.        ,  2.59420132,  3.36494024,  4.3646662 ,  5.6614114 ,
        7.34342046,  9.52515552, 12.3550855 , 16.02578954, 20.78706217,
       26.96291204, 34.97361097, 45.36429384, 58.84205542, 76.32406886,
       99.        ]), 'n_channels': 30, 'n_roi': 6, 'bold_delay': 5, 'to_many': True, 'random_subsample': True, 'sample_per_epoch': 2048, 'WINDOW_SIZE': 2048, 'optimizer': 'adam', 'lr': 0.0003, 'weight_decay': 0.0001, 'batch_size': 4, 'preproc_type': 'dB_log', 'loss_function': 'mse', 'model_type': '1D_CNN_AE_wav2vec2'}
AutoEncoder1D(
  (spatial_reduce): ConvBlock(
    (conv1d): Conv1d(480, 256, kernel_size=(3,), stride=(1,), padding=same, bias=False)
    (norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    (activation): GELU()
    (d

train/loss_0,█▇▃▅▂▁▄▃▃▃▃▂▃▄▄▂▃▅▃▅▅▃▅▅▃▅▃▃▅▂▂▅▆▂▄▁▄▄▅▃
train/loss_1,▁▄▆▅▄▃▄▅▆▄▅▅▅▇▅▆▇█▇▅▅▄▆▇▅▆▆▅▅▅▆█▇▆█▇▇██▆
val/corr_mean,▁
val/loss_0,▁▁▄▅▅▄▆▅█▅▆▆▅▆▆█▆█▇▄▅▅▅▇▅▆▆▅▅▄▆▇▆▅█▇▆▆▆▆
val/loss_1,█▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss_0,0.98804
train/loss_1,0.03958
val/loss_0,1.38094
val/loss_1,-0.14069


wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


{'n_electrodes': 30, 'n_freqs': 16, 'n_channels_out': 6, 'channels': [256, 256, 256, 256], 'kernel_sizes': [3, 3, 3], 'strides': [8, 4, 4], 'dilation': [1, 1, 1], 'decoder_reduce': 4, 'dataset_name': 'CWL', 'new_fps': 100, 'freqs': array([ 2.        ,  2.59420132,  3.36494024,  4.3646662 ,  5.6614114 ,
        7.34342046,  9.52515552, 12.3550855 , 16.02578954, 20.78706217,
       26.96291204, 34.97361097, 45.36429384, 58.84205542, 76.32406886,
       99.        ]), 'n_channels': 30, 'n_roi': 6, 'bold_delay': 5, 'to_many': True, 'random_subsample': True, 'sample_per_epoch': 2048, 'WINDOW_SIZE': 2048, 'optimizer': 'adam', 'lr': 0.0003, 'weight_decay': 0.0001, 'batch_size': 4, 'preproc_type': 'dB_log', 'loss_function': 'mse', 'model_type': '1D_CNN_AE_wav2vec2'}
AutoEncoder1D(
  (spatial_reduce): ConvBlock(
    (conv1d): Conv1d(480, 256, kernel_size=(3,), stride=(1,), padding=same, bias=False)
    (norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    (activation): GELU()
    (d

train/loss_0,█▃▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/loss_1,▁▇██████████████████████████████████████
val/corr_mean,▁▅▅▆▆▆▆▆▇▆█▇
val/loss_0,▃▆▅▅▄▅▆▇▅▃▆▃▅▅▇▄▄▅▇▃▄█▇▃▃▃▅▂▄▃▅▅▄▁▄▄▇▅▆▃
val/loss_1,▁▅▅▆▆▅▆▅▆▆▅▆▆▆▆▆▇▆▇▇▇▆▆▇▇█▇█▇▇▇▇▇█▇▇▆▆▆▇
train/loss_0,0.04935
train/loss_1,0.9706
val/loss_0,1.40926
val/loss_1,0.34958


wandb: wandb version 0.12.11 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


{'n_electrodes': 30, 'n_freqs': 16, 'n_channels_out': 6, 'channels': [256, 256, 256, 256], 'kernel_sizes': [3, 3, 3], 'strides': [8, 4, 4], 'dilation': [1, 1, 1], 'decoder_reduce': 4, 'dataset_name': 'CWL', 'new_fps': 100, 'freqs': array([ 2.        ,  2.59420132,  3.36494024,  4.3646662 ,  5.6614114 ,
        7.34342046,  9.52515552, 12.3550855 , 16.02578954, 20.78706217,
       26.96291204, 34.97361097, 45.36429384, 58.84205542, 76.32406886,
       99.        ]), 'n_channels': 30, 'n_roi': 6, 'bold_delay': 5, 'to_many': True, 'random_subsample': True, 'sample_per_epoch': 2048, 'WINDOW_SIZE': 2048, 'optimizer': 'adam', 'lr': 0.0003, 'weight_decay': 0.0001, 'batch_size': 4, 'preproc_type': 'dB_log', 'loss_function': 'mse', 'model_type': '1D_CNN_AE_wav2vec2'}
AutoEncoder1D(
  (spatial_reduce): ConvBlock(
    (conv1d): Conv1d(480, 256, kernel_size=(3,), stride=(1,), padding=same, bias=False)
    (norm): LayerNorm((256,), eps=1e-05, elementwise_affine=True)
    (activation): GELU()
    (d

# 